In [1]:
using Optim, ForwardDiff, LinearAlgebra, Printf

The n-variable Rosenbrock test function is

$$
f(x) = \sum_{i=1}^{N-1}[100(x_{i+1} - x_{i}^2)^2+(1-x_i)^2].
$$

Define the 3-variable Rosenbrock test function.

In [2]:
N = 3;
f3(x)= 100.0 * (x[2] - x[1]^2)^2 + (1-x[1])^2 + 100.0 * (x[3]-x[2]^2)^2+(1-x[2])^2

f3 (generic function with 1 method)

In [3]:
g(x) = ForwardDiff.gradient(f3, x)

g (generic function with 1 method)

The linear constraint is $x_1+x_2+x3 = 1$ and $2x_1+x_2 = 0.5$.

In [4]:
a1 = ones(1,3) ; b1 = 1
a2 = [2 1 0]; b2 = 0.5
A = vcat(a1,a2); b = vcat(b1,b2);

let $C = \{x | Ax = b\}$. Projection on to C is $\text{proj}_{\mathcal C} (z) = z + A^\intercal(AA^\intercal)^{-1}(b-Az)$

In [6]:
proj(z) = z + A'* ( (A*A') \ (-A*z+b))

proj (generic function with 1 method)

In [7]:
xk = zeros(3);
step = 1e-4;
xkp = xk;
for i in 1:500
    xkp = xk - step * g(xk);
    xkp = proj(xkp)
    if rem(i,100) == 0
        @printf("iteration = %d, successive error = %f\n",i, norm(xk - xkp))
    end
    xk = xkp
end
@printf("\nThe minimizer is x = (%.3f,%.3f,%.3f)\n", xk[1], xk[2], xk[3])

iteration = 100, successive error = 0.000277
iteration = 200, successive error = 0.000016
iteration = 300, successive error = 0.000001
iteration = 400, successive error = 0.000000
iteration = 500, successive error = 0.000000

The minimizer is x = (0.027,0.446,0.527)


In [8]:
xk = [];
xk = push!(xk,zeros(3));
step = 1e-4;
for i in 1:500
    xkp = xk[i] - step * g(xk[i]);
    xkp = proj(xkp)
    if rem(i,100) == 0
        @printf("iteration = %d, successive error = %f\n",i, norm(xk[i] - xkp))
    end
    push!(xk,xkp)
end

iteration = 100, successive error = 0.000277
iteration = 200, successive error = 0.000016
iteration = 300, successive error = 0.000001
iteration = 400, successive error = 0.000000
iteration = 500, successive error = 0.000000


In [10]:
Q, R = qr(A'); Z = reshape(Q[:,3],3,1);
xbar = A \ b
fz(p) = f3(xbar + Z*p)
gz(p) = Z'*g(xbar+Z*p)
p0 = [0.]
res = Optim.optimize(fz, gz, p0; inplace = false)
p = res.minimizer
xbar+Z*p

3-element Array{Float64,1}:
 0.02698245445308882
 0.4460350910938224
 0.5269824544530889